In [1]:
# dataset-ubyte to csv
    def convert(imgf, labelf, outf, n):
        f = open(imgf, "rb")
        o = open(outf, "w")
        l = open(labelf, "rb")

        f.read(16)
        l.read(8)
        images = []

        for i in range(n):
            image = [ord(l.read(1))]
            for j in range(28*28):
                image.append(ord(f.read(1)))
            images.append(image)

        for image in images:
            o.write(",".join(str(pix) for pix in image)+"\n")
        f.close()
        o.close()
        l.close()

    rawdir = "/home/aipslab1/beomsu1000/data/MNIST/raw/"
    convertdir = wdir + "mnist_test.csv"
    if os.path.exists(convertdir) == False:
        convert(rawdir + "t10k-images-idx3-ubyte", rawdir + "t10k-labels-idx1-ubyte",
            wdir + "mnist_test.csv", 10000)
        print("Convert Finished!")
    else:
        print(convertdir + ": exists")
        
    #upload saved weights
    
    fp_w2 = wdir + 'w2.conv1.weight.csv'
    fp_w2_scale = wdir + 'w2.conv1.weight_scale.csv'
    np_w2 = np.loadtxt(fp_w2, delimiter=',', dtype=float)
    np_w2_scale = np.loadtxt(fp_w2_scale, delimiter=',', dtype=float)
    
    fp_w6 = wdir + 'w6.conv2.weight.csv'
    fp_w6_scale = wdir + 'w6.conv2.weight_scale.csv'
    np_w6 = np.loadtxt(fp_w6, delimiter=',', dtype=float)
    np_w6_scale = np.loadtxt(fp_w6_scale, delimiter=',', dtype=float)
    
    fp_w10 = wdir + 'w10.conv3.weight.csv'
    fp_w10_scale = wdir + 'w10.conv3.weight_scale.csv'
    np_w10 = np.loadtxt(fp_w10, delimiter=',', dtype=float)
    np_w10_scale = np.loadtxt(fp_w10_scale, delimiter=',', dtype=float)
    
    #test verified weights
    fp_input_csv = wdir + 'mnist_test.csv'
    np_input_csv = np.loadtxt(fp_input_csv, delimiter=',', dtype=float)

    print(np_input_csv.shape)
    
    testsize = 10

    for b in range(testsize):
        _label = int(np_input_csv[b][0])
        _image = np_input_csv[b][1:785]

        # conv1
        np_ia2_q, scale_ia2 = uniform_quantize(input=_image, qbit=8)
        np_oa2 = np.zeros([32], dtype=float)
        for i in range(32):
            for j in range(784):
                np_oa2[i] += np_ia2_q[j] * np_w2[i+j]
        np_oa2 *= scale_ia2 * np_w2_scale

        #conv2
        np_ia6 = np.empty([32], dtype=float)
        for i in range(32):
            np_ia6[i] = np_oa2[i] if np_oa2[i] > 0 else 0

        np_ia6_q, scale_ia6 = uniform_quantize(input=np_ia2, qbit=8)
        np_oa6 = np.zeros([32], dtype=float)

        for i in range(32):
            for j in range(32):
                np_oa6[i] += np_ia6_q[j] * np_w6[i*32+j]
        np_oa6 *= scale_ia6 * np_w6_scale
    
        #conv3
        np_ia10 = np.empty([32], dtype=float)
        for i in range(32):
            np_ia10[i] = np_oa6[i] if np_oa6[i] > 0 else 0

        np_ia10_q, scale_ia10 = uniform_quantize(input=np_ia2, qbit=8)
        np_oa10 = np.zeros([32], dtype=float)

        for i in range(10):
            for j in range(32):
                np_oa10[i] += np_ia10_q[j] * np_w10[i*32+j]
        np_oa10 *= scale_ia10 * np_w10_scale

        _infer = np.argmax(np_oa10)
        print("inference: " + str(_infer) + ", answer: " + str(_label))

IndentationError: unexpected indent (3509870924.py, line 2)